In [ ]:
#Install the necesary libraries
!pip -q install tensorflow
!pip -q install ISR gdown
!pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

import os
if not os.path.exists('weights/rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf'):

  # download pretrained weights
  import os
  github_weights_url = 'https://github.com/AwaleSajil/ISR_weights'
  folder_name = os.path.basename(github_weights_url)
  !git clone $github_weights_url
  !mv $folder_name 'weights'
  
#Clone the repositories we are going to use
!git clone https://github.com/AwaleSajil/ISR-Youtube-


#Import the models set by the library
from ISR.models import RDN, RRDN
import numpy as np
from PIL import Image
import IPython.display

%cd /
from ghc.l_ghc_cf import l_ghc_cf
%cd content

#Set the params of the models
rrdn_gans = RRDN(arch_params={'C': 4, 'D':3, 'G':32, 'G0':32, 'x':4, 'T': 10})
rrdn_gans.model.load_weights('weights/rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf5')

large_rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
large_rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_PSNR_epoch086.hdf5')

small_rdn = RDN(arch_params={'C': 3, 'D':10, 'G':64, 'G0':64, 'x':2})
small_rdn.model.load_weights('weights/rdn-C3-D10-G64-G064-x2_PSNR_epoch134.hdf5')

large_noise_cancelling_rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
large_noise_cancelling_rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5')

#Clear output and be done with the configuration
from IPython.display import clear_output
clear_output()
print("Configuración terminada")

In [ ]:
#Set the execution of the model

#choose the model to use
choice_model = 'RDN noise cancelling (C6-D20-G64-G064-x2_ArtefactCancelling_epoch219)'

#Set the image to use
imageURL__or__pathToImage = '/content/2.jpg' #@param {type:"string"}

sourceImg = None
def path_leaf(path):
  import ntpath
  head, tail = ntpath.split(path)
  return tail or ntpath.basename(head)

try:
  fname = path_leaf(imageURL__or__pathToImage)
  sourceImg = Image.open(fname)
except FileNotFoundError:
  !curl $imageURL__or__pathToImage > 'img.png'
  fname = 'img.png'
  sourceImg = Image.open(fname)
except:
  raise Exception("Image address is incorrect")

#Set the image
sr_img = large_noise_cancelling_rdn.predict(np.array(sourceImg))
Image.fromarray(sr_img)

#Set the result image
result_img = Image.fromarray(sr_img)
result_img.save("/content/Resultados/result.png")

#Set the original image
resizedOriginal = sourceImg.resize(size=(result_img.size[0], result_img.size[1]), resample=Image.BICUBIC)
resizedOriginal.save("/content/Resultados/original.png")

!cp '/content/Resultados/result.png' '/usr/local/share/jupyter/nbextensions/result.png'
!cp '/content/Resultados/original.png' '/usr/local/share/jupyter/nbextensions/Resultados/original.png'

width, height = result_img.size

import time
from IPython.display import clear_output


with open('/content/ISR-Youtube-/index.html','r') as file:
    htm = file.read()

time.sleep(1)
html_code = htm.format(width, height)
display(IPython.display.HTML(html_code))